# Tiền xử lý dữ liệu

In [7]:
import pandas as pd
import numpy as np
import os

# Hàm gộp các file .csv
def combine_csv_files(dataset_path, file_names):
    df_list = []
    for file_name in file_names:
        file_path = os.path.join(dataset_path, file_name + '.csv')
        try:
            # Sử dụng encoding='utf-8-sig' để bỏ qua BOM
            df = pd.read_csv(file_path, encoding='utf-8-sig', low_memory=False)
            df_list.append(df)
            print(f"Đã đọc: {file_name}.csv")
        except FileNotFoundError:
            print(f"Không tìm thấy file: {file_name}.csv")
    combined_df = pd.concat(df_list, ignore_index=True)
    print(f"Kích thước DataFrame gộp: {combined_df.shape}")
    return combined_df

### Bước 1: Gộp file

In [9]:
# Định nghĩa đường dẫn và tên file
dataset = 'UNSW-NB15'
csv_file_names = ('UNSW_NB15_training-set', 'UNSW_NB15_testing-set')

# Đường dẫn của bạn
dataset_path = 'Dataset/UNSW-NB15'

# Gộp các file
unsw_nb15 = combine_csv_files(dataset_path=dataset_path, file_names=csv_file_names)

# Xem 5 dòng đầu
print("Dữ liệu ban đầu:")
unsw_nb15.head()

Đã đọc: UNSW_NB15_training-set.csv
Đã đọc: UNSW_NB15_testing-set.csv
Kích thước DataFrame gộp: (257673, 45)
Dữ liệu ban đầu:


,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.000011,udp,-,INT,2,0,496,0,90909.0902,...,1,2,0,0,0,1,2,0,Normal,0
1,2,0.000008,udp,-,INT,2,0,1762,0,125000.0003,...,1,2,0,0,0,1,2,0,Normal,0
2,3,0.000005,udp,-,INT,2,0,1068,0,200000.0051,...,1,3,0,0,0,1,3,0,Normal,0
3,4,0.000006,udp,-,INT,2,0,900,0,166666.6608,...,1,3,0,0,0,2,3,0,Normal,0
4,5,0.000010,udp,-,INT,2,0,2126,0,100000.0025,...,1,3,0,0,0,2,3,0,Normal,0


### Bước 2: Tiền xử lý dữ liệu
Chúng ta sẽ thực hiện các bước tiền xử lý từng bước nhỏ.
2.1. Kiểm tra dữ liệu thiếu và kiểu dữ liệu
Chạy đoạn code sau để kiểm tra dữ liệu thiếu và kiểu dữ liệu:

In [24]:
# Kiểm tra dữ liệu thiếu
print("Dữ liệu thiếu:")
print(unsw_nb15.isnull().sum())

# Kiểm tra kiểu dữ liệu
print("\nKiểu dữ liệu:")
print(unsw_nb15.dtypes)

# Xác nhận kích thước DataFrame
print("\nKích thước DataFrame:")
print(unsw_nb15.shape)

Dữ liệu thiếu:
id                   0
dur                  0
proto                0
service              0
state                0
spkts                0
dpkts                0
sbytes               0
dbytes               0
rate                 0
sttl                 0
dttl                 0
sload                0
dload                0
sloss                0
dloss                0
sinpkt               0
dinpkt               0
sjit                 0
djit                 0
swin                 0
stcpb                0
dtcpb                0
dwin                 0
tcprtt               0
synack               0
ackdat               0
smean                0
dmean                0
trans_depth          0
response_body_len    0
ct_srv_src           0
ct_state_ttl         0
ct_dst_ltm           0
ct_src_dport_ltm     0
ct_dst_sport_ltm     0
ct_dst_src_ltm       0
is_ftp_login         0
ct_ftp_cmd           0
ct_flw_http_mthd     0
ct_src_ltm           0
ct_srv_dst           0
is_sm_ips_ports    

### Bước 3: Kiểm tra giá trị bất thường và mã hóa cột phân loại
Kiểm tra giá trị bất thường (như Infinity, -Infinity) trong các cột số. Mã hóa các cột phân loại (proto, service, state, attack_cat).

In [26]:
# Kiểm tra giá trị vô cực (Infinity, -Infinity)
numeric_cols = unsw_nb15.select_dtypes(include=[np.number]).columns
inf_check = unsw_nb15[numeric_cols].replace([np.inf, -np.inf], np.nan).isnull().sum()
print("Số giá trị vô cực (Infinity) trong các cột số:")
print(inf_check[inf_check > 0])

# Mã hóa các cột phân loại
from sklearn.preprocessing import LabelEncoder

# Các cột cần mã hóa
categorical_columns = ['proto', 'service', 'state', 'attack_cat']

# Mã hóa từng cột và in ánh xạ
for col in categorical_columns:
    le = LabelEncoder()
    unsw_nb15[col] = le.fit_transform(unsw_nb15[col])
    print(f"\nÁnh xạ cho cột {col}:")
    print(dict(zip(le.classes_, le.transform(le.classes_))))

# Xem lại 5 dòng đầu sau khi mã hóa
print("\nDữ liệu sau khi mã hóa:")
unsw_nb15.head()

Số giá trị vô cực (Infinity) trong các cột số:
Series([], dtype: int64)

Ánh xạ cho cột proto:
{'3pc': 0, 'a/n': 1, 'aes-sp3-d': 2, 'any': 3, 'argus': 4, 'aris': 5, 'arp': 6, 'ax.25': 7, 'bbn-rcc': 8, 'bna': 9, 'br-sat-mon': 10, 'cbt': 11, 'cftp': 12, 'chaos': 13, 'compaq-peer': 14, 'cphb': 15, 'cpnx': 16, 'crtp': 17, 'crudp': 18, 'dcn': 19, 'ddp': 20, 'ddx': 21, 'dgp': 22, 'egp': 23, 'eigrp': 24, 'emcon': 25, 'encap': 26, 'etherip': 27, 'fc': 28, 'fire': 29, 'ggp': 30, 'gmtp': 31, 'gre': 32, 'hmp': 33, 'i-nlsp': 34, 'iatp': 35, 'ib': 36, 'icmp': 37, 'idpr': 38, 'idpr-cmtp': 39, 'idrp': 40, 'ifmp': 41, 'igmp': 42, 'igp': 43, 'il': 44, 'ip': 45, 'ipcomp': 46, 'ipcv': 47, 'ipip': 48, 'iplt': 49, 'ipnip': 50, 'ippc': 51, 'ipv6': 52, 'ipv6-frag': 53, 'ipv6-no': 54, 'ipv6-opts': 55, 'ipv6-route': 56, 'ipx-n-ip': 57, 'irtp': 58, 'isis': 59, 'iso-ip': 60, 'iso-tp4': 61, 'kryptolan': 62, 'l2tp': 63, 'larp': 64, 'leaf-1': 65, 'leaf-2': 66, 'merit-inp': 67, 'mfe-nsp': 68, 'mhrp': 69, 'micp': 70,

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.000011,119,0,5,2,0,496,0,90909.0902,...,1,2,0,0,0,1,2,0,6,0
1,2,0.000008,119,0,5,2,0,1762,0,125000.0003,...,1,2,0,0,0,1,2,0,6,0
2,3,0.000005,119,0,5,2,0,1068,0,200000.0051,...,1,3,0,0,0,1,3,0,6,0
3,4,0.000006,119,0,5,2,0,900,0,166666.6608,...,1,3,0,0,0,2,3,0,6,0
4,5,0.000010,119,0,5,2,0,2126,0,100000.0025,...,1,3,0,0,0,2,3,0,6,0


### Bước 4: Chuẩn hóa đặc trưng (Normalization) và kiểm tra phân phối nhãn
Bây giờ dữ liệu đã sạch (không thiếu, không có giá trị vô cực, các cột phân loại đã mã hóa), chúng ta sẽ:

Chuẩn hóa các đặc trưng số (đưa về cùng thang đo, cần thiết cho các mô hình máy học như SVM, Neural Network).
Kiểm tra phân phối nhãn (label và attack_cat) để xem dữ liệu có mất cân bằng không.

In [30]:
from sklearn.preprocessing import StandardScaler

# Chọn các cột đặc trưng (loại bỏ cột không cần thiết như id, label, attack_cat)
features = unsw_nb15.drop(columns=['id', 'label', 'attack_cat'])

# Chuẩn hóa đặc trưng
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# Chuyển lại thành DataFrame
scaled_df = pd.DataFrame(scaled_features, columns=features.columns)

# Thêm lại cột label và attack_cat vào DataFrame đã chuẩn hóa
final_df = pd.concat([scaled_df, unsw_nb15[['label', 'attack_cat']].reset_index(drop=True)], axis=1)

# Kiểm tra phân phối nhãn
print("Phân phối của cột label:")
print(unsw_nb15['label'].value_counts())

print("\nPhân phối của cột attack_cat:")
print(unsw_nb15['attack_cat'].value_counts())

# Xem 5 dòng đầu của dữ liệu đã chuẩn hóa
print("\nDữ liệu sau khi chuẩn hóa:")
final_df.head()

Phân phối của cột label:
label
1    164673
0     93000
Name: count, dtype: int64

Phân phối của cột attack_cat:
attack_cat
6    93000
5    58871
3    44525
4    24246
2    16353
7    13987
0     2677
1     2329
8     1511
9      174
Name: count, dtype: int64

Dữ liệu sau khi chuẩn hóa:


,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,label,attack_cat
0,-0.208678,0.415177,-0.692762,0.742077,-0.130765,-0.165331,-0.046480,-0.098409,-0.002151,0.722026,...,-0.520051,-0.568574,-0.110419,-0.110372,-0.193597,-0.690790,-0.654825,-0.120335,0,6
1,-0.208679,0.415177,-0.692762,0.742077,-0.130765,-0.165331,-0.039194,-0.098409,0.210460,0.722026,...,-0.520051,-0.568574,-0.110419,-0.110372,-0.193597,-0.690790,-0.654825,-0.120335,0,6
2,-0.208679,0.415177,-0.692762,0.742077,-0.130765,-0.165331,-0.043188,-0.098409,0.678204,0.722026,...,-0.520051,-0.478652,-0.110419,-0.110372,-0.193597,-0.690790,-0.562869,-0.120335,0,6
3,-0.208679,0.415177,-0.692762,0.742077,-0.130765,-0.165331,-0.044155,-0.098409,0.470318,0.722026,...,-0.520051,-0.478652,-0.110419,-0.110372,-0.193597,-0.571689,-0.562869,-0.120335,0,6
4,-0.208678,0.415177,-0.692762,0.742077,-0.130765,-0.165331,-0.037100,-0.098409,0.054546,0.722026,...,-0.520051,-0.478652,-0.110419,-0.110372,-0.193597,-0.571689,-0.562869,-0.120335,0,6


In [32]:
# Lưu dữ liệu đã xử lý thành file CSV
final_df.to_csv('Dataset/dataset_combined/UNSW-NB15.csv', index=False)
print("Dữ liệu đã được lưu vào: Dataset/dataset_combined/UNSW-NB15.csv")

# Kiểm tra 5 dòng đầu của dữ liệu đã lưu
print("\nXác nhận dữ liệu đã lưu:")
pd.read_csv('Dataset/dataset_combined/UNSW-NB15.csv').head()

Dữ liệu đã được lưu vào: C:/Users/garan/Dataset/dataset_combined/UNSW-NB15.csv

Xác nhận dữ liệu đã lưu:


,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,label,attack_cat
0,-0.208678,0.415177,-0.692762,0.742077,-0.130765,-0.165331,-0.046480,-0.098409,-0.002151,0.722026,...,-0.520051,-0.568574,-0.110419,-0.110372,-0.193597,-0.690790,-0.654825,-0.120335,0,6
1,-0.208679,0.415177,-0.692762,0.742077,-0.130765,-0.165331,-0.039194,-0.098409,0.210460,0.722026,...,-0.520051,-0.568574,-0.110419,-0.110372,-0.193597,-0.690790,-0.654825,-0.120335,0,6
2,-0.208679,0.415177,-0.692762,0.742077,-0.130765,-0.165331,-0.043188,-0.098409,0.678204,0.722026,...,-0.520051,-0.478652,-0.110419,-0.110372,-0.193597,-0.690790,-0.562869,-0.120335,0,6
3,-0.208679,0.415177,-0.692762,0.742077,-0.130765,-0.165331,-0.044155,-0.098409,0.470318,0.722026,...,-0.520051,-0.478652,-0.110419,-0.110372,-0.193597,-0.571689,-0.562869,-0.120335,0,6
4,-0.208678,0.415177,-0.692762,0.742077,-0.130765,-0.165331,-0.037100,-0.098409,0.054546,0.722026,...,-0.520051,-0.478652,-0.110419,-0.110372,-0.193597,-0.571689,-0.562869,-0.120335,0,6


In [34]:
# Lưu dữ liệu đã làm sạch (chưa chuẩn hóa) thành file CSV
unsw_nb15.to_csv('Dataset/dataset_cleaned/UNSW-NB15.csv', index=False)
print("Dữ liệu đã được lưu vào: Dataset/dataset_cleaned/UNSW-NB15.csv")

# Kiểm tra 5 dòng đầu của dữ liệu đã lưu
print("\nXác nhận dữ liệu cleaned:")
pd.read_csv('Dataset/dataset_cleaned/UNSW-NB15.csv').head()

Dữ liệu đã được lưu vào: C:/Users/garan/Dataset/dataset_cleaned/UNSW-NB15.csv

Xác nhận dữ liệu cleaned:


,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.000011,119,0,5,2,0,496,0,90909.0902,...,1,2,0,0,0,1,2,0,6,0
1,2,0.000008,119,0,5,2,0,1762,0,125000.0003,...,1,2,0,0,0,1,2,0,6,0
2,3,0.000005,119,0,5,2,0,1068,0,200000.0051,...,1,3,0,0,0,1,3,0,6,0
3,4,0.000006,119,0,5,2,0,900,0,166666.6608,...,1,3,0,0,0,2,3,0,6,0
4,5,0.000010,119,0,5,2,0,2126,0,100000.0025,...,1,3,0,0,0,2,3,0,6,0


In [42]:
import pandas as pd

# Đọc dữ liệu đã xử lý
data = pd.read_csv('Dataset/dataset_combined/UNSW-NB15.csv')

# Kiểm tra phân phối lớp theo attack_cat (tương tự LABEL trong CIC-IDS2017)
print("Class distribution:")
print(data['attack_cat'].value_counts())

# Kiểm tra phân phối chuẩn hóa (normalized)
print("\nClass distribution (normalized):")
print(data['attack_cat'].value_counts(normalize=True) * 100)

Class distribution:
attack_cat
6    93000
5    58871
3    44525
4    24246
2    16353
7    13987
0     2677
1     2329
8     1511
9      174
Name: count, dtype: int64

Class distribution (normalized):
attack_cat
6    36.092256
5    22.847175
3    17.279653
4     9.409601
2     6.346416
7     5.428198
0     1.038914
1     0.903859
8     0.586402
9     0.067527
Name: proportion, dtype: float64


### Bước 5: áp dụng SMOTE để cân bằng dữ liệu (vì dữ liệu hiện tại mất cân bằng). Sau đó, bạn có thể dùng dữ liệu đã cân bằng để huấn luyện mô hình.

In [36]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

# Đọc dữ liệu đã chuẩn hóa
data = pd.read_csv('Dataset/dataset_combined/UNSW-NB15.csv')

# Tách đặc trưng (X) và nhãn (y)
X = data.drop(columns=['label', 'attack_cat'])
y = data['label']  # Có thể dùng 'attack_cat' nếu bạn muốn phân loại đa lớp

# Chia tập huấn luyện và kiểm tra (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Áp dụng SMOTE để cân bằng dữ liệu trên tập huấn luyện
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Kiểm tra phân phối nhãn sau khi áp dụng SMOTE
print("Phân phối nhãn trên tập huấn luyện sau SMOTE:")
print(pd.Series(y_train_smote).value_counts())

# Kích thước tập huấn luyện và kiểm tra
print("\nKích thước tập huấn luyện sau SMOTE:", X_train_smote.shape)
print("Kích thước tập kiểm tra:", X_test.shape)

Phân phối nhãn trên tập huấn luyện sau SMOTE:
label
1    131738
0    131738
Name: count, dtype: int64

Kích thước tập huấn luyện sau SMOTE: (263476, 42)
Kích thước tập kiểm tra: (51535, 42)


Lưu dữ liệu cần thiết từ notebook hiện tại

In [44]:
# Lưu dữ liệu huấn luyện và kiểm tra
X_train_smote.to_csv('Dataset/dataset_combined/UNSW_NB15_X_train_smote.csv', index=False)
y_train_smote.to_csv('Dataset/dataset_combined/UNSW_NB15_y_train_smote.csv', index=False)
X_test.to_csv('Dataset/dataset_combined/UNSW_NB15_X_test.csv', index=False)
y_test.to_csv('Dataset/dataset_combined/UNSW_NB15y_test.csv', index=False)

print("Dữ liệu huấn luyện và kiểm tra đã được lưu vào thư mục: Dataset/dataset_combined/")

Dữ liệu huấn luyện và kiểm tra đã được lưu vào thư mục: C:/Users/garan/Dataset/dataset_combined/
